# EEG preprocessing 

## Imports

The data will be processed using the mne library. Also there are libraries made in eegyolk in order to load the metadata, eeg data and the event markers. Those libraries need to be imported

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
from eegyolk import helper_functions as hf # library useful for eeg and erp data cleaning
from eegyolk import initialization_functions #library to import data
from eegyolk import epod_helper

## Load metadata and eeg files

First the different pathways for the different datasets need to be defined. There are three pathways: eeg, metadata and events. The files can be loaded using the initialization_functions library. All event markers needs to be saved in a seperate folder. If not saved already, the event markers will be saved using the initialization_function library. 

In [2]:
# define pathways
# path_eeg = os.path.join("F:", os.sep, "Stage", "ePODIUM", "Data", "not_zip")
# path_metadata = os.path.join("F:", os.sep, "Stage", "ePODIUM", "Data", "metadata")
# path_eventmarkers = os.path.join("F:", os.sep, "Stage", "ePODIUM", "Data", "not_zip", "event_markers")

path_metadata = os.path.join('../epod_data_not_pushed','metadata')
#path_excelFiles = 'C:/Projects/new_data_karin_EEG/metadata/'
path_eeg = os.path.join('../epod_data_not_pushed','not_zip')
path_eventmarkers = os.path.join('../epod_data_not_pushed','not_zip', 'event_markers')

In [3]:
# test to be removed
os.listdir(path_eeg)

['101to110', '121to130', '131to140', '141to150', '151to160', 'event_markers']

In [4]:
# load metadata and eeg
files_metadata = ["children.txt", "cdi.txt", "parents.txt", "CODES_overview.txt"]  
children, cdi, parents, codes = initialization_functions.load_metadata(path_metadata, files_metadata)
# load metadata

In [5]:
# eeg, eeg_filenames, eeg_filepaths = initialization_functions.load_dataset(path_eeg) # load eeg

In [6]:
# # load events 
# events_files = os.listdir(path_eventmarkers)
# if len(events_files) == 0 or path_eventmarkers == False: # check if event markers are saved in a seperate folder
#     initialization_functions.save_event_markers(path_eventmarkers, eeg, eeg_filenames) # save event markers
    
# event_markers = initialization_functions.load_event_markers(path_eventmarkers, eeg_filenames) # load event markers
# event_markers_simplified = epod_helper.group_event_markers(event_markers) # simplify events

In [8]:
path_eventmarkers2= 'love'
events_files = os.listdir(path_eventmarkers2)
if not events_files:
    initialization_functions.caller_save_event_markers(path_eventmarkers2, initialization_functions.generator_load_dataset(path_eeg))
        

99 EEG files loaded


## Data info

Choose which participant you want to view in the box below. 

In [ ]:
index = widgets.IntText(
    step=0.1,
    description='Participant',
    disabled=False
)
widgets.VBox([index])

In [ ]:
index = int(index.value)

In [ ]:
print("\n", eeg[index].info)

In [ ]:
# plot of the used sensors
montage = mne.channels.make_standard_montage('standard_1020')
#montage.plot(kind='topomap', show_names=True)
eeg[index].info.set_montage(montage, on_missing='ignore')
eeg[index].plot_sensors(ch_type='eeg', show_names=True)

In [ ]:
eeg[index].plot()

In [ ]:
%matplotlib inline 
fig = mne.viz.plot_events(event_markers_simplified[index], event_id = epod_helper.event_dictionary)

## Filtering

In [ ]:
# bad channel remover
def removebadchannel(eeg):
    for i in range(len(eeg)):
        if len(eeg[i].info['bads']) != 0:
            eeg[index] = mne.pick_types(eeg[i].info, meg=False, eeg=True, exclude='bads')
    return eeg
removebadchannel(eeg)

In [ ]:
lowpass = widgets.IntText(
    step=0.1,
    description='lowpass:',
    disabled=False
)

highpass = widgets.IntText(
    step=0.1,
    description='highpass:',
    disabled=False
)

widgets.VBox([lowpass,highpass])


In [ ]:
# change type to integer
lowpass = int(lowpass.value)
highpass = int(highpass.value)

Bandpass filter doesn't work due to memory issues. Can be fixed with preload = True in the load_data() in helper_functions.py. However, jupyter crashes due to lack of memory when done. Needs to be fixed.

In [ ]:
# bandpass filter
for index in range(len(eeg)): eeg[index] = hf.band_pass_filter(eeg[index], lowpass, highpass)

Below a working filter, but not imported from helper_functions.py. This is a band pass filter with defined frequency. The filter is useful for limitting the bandwidth of the output signal to avoid noise. 

In [ ]:
# plotting filter
filter_params = mne.filter.create_filter(eeg[index].get_data(), eeg[index].info['sfreq'],
                                         l_freq=lowpass, h_freq=highpass)
mne.viz.plot_filter(filter_params, eeg[index].info['sfreq'], flim=(0.01, 5))

The data contains power line noise which disrupts the rest of the signal. The notch filter filters out the fequency of 50 Hz. Does not work because of the preload memory issue. 

In [ ]:
# create notch filter
freqs = (60, 120, 180, 240)
for i in range(len(eeg)): eeg[i] = eeg[i].notch_filter(freqs=freqs)

## Creating epoched data

Epochs are created with joining the eeg data with a specific event. tmin and tmax are the start and stop time relative to each event. mne.Epochs automaticaly create a baseline correction.

In [ ]:
def create_epochs(eeg, event_markers_simplified):
    epochs =  []
    for i in range(len(eeg)): 
        single_epoch = mne.Epochs(eeg[i], event_markers_simplified[i], tmin=-0.3, tmax=0.7)
        epochs.append(single_epoch)
    return epochs

epochs = create_epochs(eeg, event_markers_simplified)

In [ ]:
print(epochs)

In [ ]:
event_dictionary = epod_helper.event_dictionary
event_dictionary

In [ ]:
def evoked_responses(epochs, event_dictionary):
    evoked = []
    for i in range(len(epod_helper.event_dictionary)):
        avg_epoch = epochs[i].average()
        evoked.append(avg_epoch)
    return evoked

evoked = evoked_responses(epochs, event_dictionary)
        
#evoked.plot()
#print(([key for key in epod_helper.event_dictionary.keys()][i], [value for value in epod_helper.event_dictionary.values()][i]))
        

In [ ]:
#evoked[2].plot(spatial_colors=True)
#print(([key for key in epod_helper.event_dictionary.keys()][i], [value for value in epod_helper.event_dictionary.values()][i]))

## Create pandas dataset out of epoched data

In [ ]:
epochs = epochs[0:4] # subset to test function
def create_pd_df(epochs):
    df_epochs = pd.DataFrame()

    for i in range(len(epochs)):
        df = epochs[i].to_data_frame()
        df['index'] = i
        df_epochs = df_epochs.append(df)
    return df_epochs

create_pd_df(epochs)

In [ ]:
df_epochs